In [1]:
import os; os.chdir("../")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
record = []
dfs = []
for f in os.walk("results"):
    if len(f[-1]) > 0:
        info = f[0]
        if len(info.split("/")) < 5:
            continue
        _, dataset, model, framework, explainer = info.split("/")

        abpc = np.load(info + "/abpc.npy").mean()
        cmpx = np.load(info + "/cmpx.npy").mean()
        cmpd = np.load(info + "/cmpd.npy").mean()
        
        record.append([dataset, model, framework, explainer, abpc, cmpx, cmpd])
        print(f"Dataset: {dataset}, Model: {model}, Framework: {framework}, Explainer: {explainer}, ABPC: {abpc}, CMPX: {cmpx}, CMPD: {cmpd}")

record = pd.DataFrame(record, columns=["dataset", "model", "framework", "explainer", "abpc", "cmpx", "cmpd"])

Dataset: Wine Quality, Model: xgb, Framework: pnpxai, Explainer: shap, ABPC: 0.06460812037523284, CMPX: 1.8191803300380707, CMPD: -0.31214957535266874
Dataset: Wine Quality, Model: xgb, Framework: pnpxai, Explainer: lime, ABPC: 0.0558186315005878, CMPX: 0.6596412932872773, CMPD: -0.08727335512638092
Dataset: Wine Quality, Model: xgb, Framework: omnixai, Explainer: shap, ABPC: 0.06542741584446049, CMPX: 1.7332260602298968, CMPD: -0.294303278028965
Dataset: Wine Quality, Model: xgb, Framework: omnixai, Explainer: lime, ABPC: 0.044880455271959364, CMPX: 1.8059902090590791, CMPD: -0.32529367752373217
Dataset: Wine Quality, Model: tab_resnet, Framework: pnpxai, Explainer: vg, ABPC: 0.034013718366622925, CMPX: 1.7201935052871704, CMPD: -0.3168277144432068
Dataset: Wine Quality, Model: tab_resnet, Framework: pnpxai, Explainer: shap, ABPC: 0.1056765974353766, CMPX: 1.7822198235988618, CMPD: -0.27190269008278845
Dataset: Wine Quality, Model: tab_resnet, Framework: pnpxai, Explainer: sg, ABPC: -

In [ ]:
# record = []
# dfs = []
# for f in os.walk("results"):
#     if len(f[-1]) > 0:
#         info = f[0]
#         if "pnpxai" in info:
#             res = info.split("/")
#             if len(res) == 4:
#                 _, dataset, model, framework = info.split("/")
#             else:
#                 continue
#         else:
#             _, dataset, model, framework, explainer = info.split("/")

#         if framework == "pnpxai":
#             df = pd.read_csv(info + "/evaluation.csv")
#             df['dataset'] = dataset
#             df['model'] = model
#             df['framework'] = framework
#             dfs.append(df)
#         else:
#             abpc = np.load(info + "/abpc.npy").mean()
#             cmpx = np.load(info + "/cmpx.npy").mean()
#             cmpd = np.load(info + "/cmpd.npy").mean()
            
#             record.append([dataset, model, framework, explainer, abpc, cmpx, cmpd])
#             print(f"Dataset: {dataset}, Model: {model}, Framework: {framework}, Explainer: {explainer}, ABPC: {abpc}, CMPX: {cmpx}, CMPD: {cmpd}")

# df = pd.concat(dfs, ignore_index=True).set_index(['dataset', 'model', 'framework', 'explainer', 'metric'])['value'].unstack().reset_index()
# record = pd.DataFrame(record, columns=["dataset", "model", "framework", "explainer", "abpc", "cmpx", "cmpd"])
# record = pd.concat([record, df], axis=0).reset_index(drop=True)

Dataset: Wine Quality, Model: xgb, Framework: omnixai, Explainer: shap, ABPC: 0.06542741584446049, CMPX: 1.7332260602298968, CMPD: -0.294303278028965
Dataset: Wine Quality, Model: xgb, Framework: omnixai, Explainer: lime, ABPC: 0.044880455271959364, CMPX: 1.8059902090590791, CMPD: -0.32529367752373217
Dataset: Wine Quality, Model: tab_resnet, Framework: openxai, Explainer: shap, ABPC: 0.12462404012680053, CMPX: 1.7170379149913788, CMPD: -0.24370835736393928
Dataset: Wine Quality, Model: tab_resnet, Framework: openxai, Explainer: sg, ABPC: -0.00912669402983738, CMPX: 1.8766690516471862, CMPD: -0.38263517588377
Dataset: Wine Quality, Model: tab_resnet, Framework: openxai, Explainer: itg, ABPC: 0.1193992180004716, CMPX: 1.7307261663675308, CMPD: -0.2506258637458086
Dataset: Wine Quality, Model: tab_resnet, Framework: openxai, Explainer: grad, ABPC: -0.009568325662403367, CMPX: 1.866032017469406, CMPD: -0.38086107224225996
Dataset: Wine Quality, Model: tab_resnet, Framework: openxai, Expla

In [8]:
EXP_MAP = {
    "shap" : "KernelSHAP",
    "kernel_shap" : "KernelSHAP",
    "lime" : "LIME",
    "integrated_gradients" : "Integrated Gradients",
    "itg": "Integrated Gradients",
    "grad": "Gradient",
    "gradient": "Gradient",
    "lrp_uniform_epsilon" : "LRP",
    "smooth_grad" : "SmoothGrad",
    "sg" : "SmoothGrad",
    "var_grad" : "VarGrad",
    "grad_x_input" : "Gradient x Input",
    "ig" : "Gradient x Input",
}

MODEL_MAP = {
    "xgb" : "XGBoost",
    "tab_resnet": "Neural Networks (NN)",
}

FRAMEWORK_MAP = {
    "omnixai" : "OmniXAI",
    "openxai": "OpenXAI",
    "pnpxai" : "PnPXAI",
}
\
NAME_MAP = EXP_MAP.copy()
NAME_MAP.update(MODEL_MAP)
NAME_MAP.update(FRAMEWORK_MAP)

In [9]:
performance = record.replace(NAME_MAP).drop(columns='dataset').sort_values(by=["model", "explainer", "framework"]).reset_index(drop=True)

In [10]:
performance

,model,framework,explainer,abpc,cmpx,cmpd
0,Neural Networks (NN),OpenXAI,Gradient,-0.009568,1.866032,-0.380861
1,Neural Networks (NN),PnPXAI,Gradient,-0.009568,1.720194,-0.372879
2,Neural Networks (NN),OpenXAI,Gradient x Input,-0.007871,1.892230,-0.384743
3,Neural Networks (NN),PnPXAI,Gradient x Input,0.119399,2.019815,-0.344283
4,Neural Networks (NN),OpenXAI,Integrated Gradients,0.119399,1.730726,-0.250626
5,Neural Networks (NN),PnPXAI,Integrated Gradients,0.000000,0.000000,0.000000
6,Neural Networks (NN),OpenXAI,KernelSHAP,0.124624,1.717038,-0.243708
7,Neural Networks (NN),PnPXAI,KernelSHAP,0.105677,1.782220,-0.271903
8,Neural Networks (NN),OpenXAI,LIME,-0.009270,1.881029,-0.383622
9,Neural Networks (NN),PnPXAI,LIME,0.080563,0.729816,-0.081513


In [ ]:
STOP

# Debugging

In [ ]:
fms = ['openxai', 'pnpxai']
exps = ['grad', 'ig']

res = {}
for fm in fms:
    for exp in exps:
        path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/explanations.npy"
        res[(fm, exp)] = np.load(path)


In [ ]:
res[("openxai", "grad")][0], res[("pnpxai", "grad")][0]

(array([-0.0195922 ,  0.04447692,  0.00061291, -0.06784541,  0.00014911,
        -0.05453618,  0.0386231 ,  0.03956512, -0.02250276, -0.03526237,
        -0.07910307], dtype=float32),
 array([-0.0195922 ,  0.04447691,  0.00061291, -0.06784542,  0.00014911,
        -0.05453619,  0.03862309,  0.03956513, -0.02250276, -0.03526237,
        -0.0791031 ]))

In [ ]:
fm = "pnpxai"
exp = "ig"
path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/explanations.npy"
a = np.load(path)

In [ ]:
a.shape

(11,)

In [ ]:
fms = ['openxai', 'pnpxai']
exps = ['grad', 'shap']

res = {}
for fm in fms:
    for exp in exps:
        path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/abpc.npy"
        res[(fm, exp)] = np.load(path)

In [ ]:
res[("openxai", "shap")][0], res[("pnpxai", "shap")][0]

(0.07961373776197433, 0.0607578344643116)

In [ ]:
res[("openxai", "shap")].mean(), res[("pnpxai", "shap")].mean()

(0.12462404012680053, 0.1056765974353766)

In [ ]:
res[("openxai", "grad")][0], res[("pnpxai", "grad")][0]

(-0.0360502190887928, -0.0360502190887928)

In [ ]:
res[("openxai", "grad")].mean(), res[("pnpxai", "grad")].mean()

(-0.009568325662403367, -0.009568325662403367)